### Whole-History-Ranking

Compute and assign player's most recent WHR rating

In [1]:
import pandas as pd

games_df = pd.read_csv("../../data/processed/data.csv")

games_df.sort_values(by=['date'], inplace=True, ignore_index=True)
games_df

,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0
...,...,...,...,...,...,...,...,...
101404,Jin Siyoung,Choi Kyeongho,w,b,2022-02-19,8p,5p,6.5
101405,Kang Dongyun,Seol Hyunjun,b,w,2022-02-19,9p,7p,6.5
101406,Kim Jiseok,Shin Jaeweon,w,b,2022-02-19,9p,4p,6.5
101407,Kim Changhoon,Kwak Wonkeun,b,w,2022-02-19,5p,3p,6.5


In [2]:
from whr import whole_history_rating
from tqdm import tqdm
import datetime

whr = whole_history_rating.Base({'w2':14, 'uncased':True})
start_date = datetime.datetime(1941, 5, 18)
for idx, row in tqdm(games_df.iterrows()):
    date = datetime.datetime.strptime(row['date'], '%Y-%m-%d')
    delta = date - start_date
    if row['win_color'] == 'b':
        whr.create_game(row['win_name'], row['lose_name'], 'B', delta.days, 0)
    else:
        whr.create_game(row['lose_name'], row['win_name'], 'W', delta.days, 0)

whr.auto_iterate()

101409it [00:33, 3033.69it/s]


(20, False)

In [3]:

games_df['WWHR'], games_df['WWHR-Error'], games_df['LWHR'], games_df['LWHR-Error'] =0,0,0,0
for player in tqdm(games_df.win_name.unique()):
    winner_df = games_df.loc[games_df.win_name == player].copy()

    loser_df = games_df.loc[games_df.lose_name == player].copy()

    rating_map, uncertainty_map = {}, {}
    for days, rating, uncertainty in whr.ratings_for_player(player):   

        game_date = datetime.datetime(1941, 5, 18) + datetime.timedelta(days=days)
        rating_map[game_date.strftime("%Y-%m-%d")] = rating
        uncertainty_map[game_date.strftime("%Y-%m-%d")] = uncertainty
        
    
    winner_df['WWHR'] = winner_df['date'].map(rating_map).shift()
    winner_df['WWHR-Error'] = winner_df['date'].map(uncertainty_map).shift()

    loser_df['LWHR'] = loser_df['date'].map(rating_map).shift()
    loser_df['LWHR-Error'] = loser_df['date'].map(uncertainty_map).shift()

    games_df.update(winner_df)
    games_df.update(loser_df)
    

100%|██████████| 1827/1827 [03:30<00:00,  8.67it/s]


In [4]:
games_df

,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi,WWHR,WWHR-Error,LWHR,LWHR-Error
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0,0.0,0.0,0.0,0.0
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0,0.0,0.0,0.0,0.0
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0,-63.0,51.0,-84.0,45.0
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0,-84.0,44.0,-63.0,51.0
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0,-63.0,51.0,-85.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...
101404,Jin Siyoung,Choi Kyeongho,w,b,2022-02-19,8p,5p,6.5,182.0,21.0,11.0,18.0
101405,Kang Dongyun,Seol Hyunjun,b,w,2022-02-19,9p,7p,6.5,324.0,11.0,334.0,13.0
101406,Kim Jiseok,Shin Jaeweon,w,b,2022-02-19,9p,4p,6.5,394.0,10.0,45.0,17.0
101407,Kim Changhoon,Kwak Wonkeun,b,w,2022-02-19,5p,3p,6.5,171.0,20.0,100.0,30.0
